In [6]:
import pandas as pd

# Read the CSV dataset into a pandas DataFrame
df = pd.read_csv('ADS0423.csv')

# Identify the dependent variable column
dependent_var = 'Highest Injury Severity'


In [7]:
# Drop rows with missing values in the dependent variable column
df = df.dropna(subset=[dependent_var])


In [8]:
# Replace 'Y' with 1 and blanks with 0 in "SV Contact Area - Rear Left" column
df['SVc_RL'] = df['SV Contact Area - Rear Left'].replace({'Y': 1, '': 0})


In [9]:
# Replace 'Y' with 1 and blanks with 0 in "SV Contact Area - Rear" column
df['SVc_R'] = df['SV Contact Area - Rear'].replace({'Y': 1, '': 0})

In [10]:
# Replace 'Y' with 1 and blanks with 0 in "SV Contact Area - Rear Right" column
df['SVc_RR'] = df['SV Contact Area - Rear Right'].replace({'Y': 1, '': 0})

In [11]:
# Replace 'Y' with 1 and blanks with 0 in "SV Contact Area - Front Left" column
df['SVc_FL'] = df['SV Contact Area - Front Left'].replace({'Y': 1, '': 0})

# Replace 'Y' with 1 and blanks with 0 in "SV Contact Area - Front" column
df['SVc_F'] = df['SV Contact Area - Front'].replace({'Y': 1, '': 0})

# Replace 'Y' with 1 and blanks with 0 in "SV Contact Area - Front Right" column
df['SVc_FR'] = df['SV Contact Area - Front Right'].replace({'Y': 1, '': 0})

In [19]:
# Replace 'Y' cells with 1
df['SV Contact Area - Left'] = df['SV Contact Area - Left'].replace('Y', 1)
df['SV Contact Area - Right'] = df['SV Contact Area - Right'].replace('Y', 1)


# Replace blank values with 0
df['SV Contact Area - Left'] = df['SV Contact Area - Left'].replace(' ', 0)
df['SV Contact Area - Right'] = df['SV Contact Area - Right'].replace(' ', 0)

# Convert the columns to numeric
df['SV Contact Area - Left'] = pd.to_numeric(df['SV Contact Area - Left'])
df['SV Contact Area - Right'] = pd.to_numeric(df['SV Contact Area - Right'])

# Create a new variable based on the summation of "SV Contact Area - Left" and "SV Contact Area - Right"
def new_var(row):
    if row['SV Contact Area - Left'] + row['SV Contact Area - Right'] >= 1:
        return 1
    else:
        return 0

df['SVc_Sides'] = df.apply(new_var, axis=1)


In [21]:
def weather_condition(row):
    if row['Weather - Clear'] == 'Y':
        return 0
    elif row['Weather - Cloudy'] == 'Y':
        return 1
    elif row['Weather - Rain'] == 'Y':
        return 2
    else:
        return 0

df['Weather'] = df.apply(weather_condition, axis=1)
df['Weather'] = df['Weather'].fillna(0).astype(int)


In [23]:
def lighting_to_numeric(row):
    if row['Lighting'] == 'Daylight' or row['Lighting'] == 'Dawn / Dusk':
        return 0
    elif row['Lighting'] == 'Dark - Lighted' or row['Lighting'] == 'Dark - Unknown Lighting':
        return 1
    elif row['Lighting'] == 'Dark - Not Lighted':
        return 2
    else:
        return 99

df['Lighting_recode'] = df.apply(lighting_to_numeric, axis=1)


In [24]:
def new_var(row):
    if row['SV Pre-Crash Movement'] == "Proceeding Straight":
        return 0
    elif row['SV Pre-Crash Movement'] == "Stopped":
        return 1
    elif row['SV Pre-Crash Movement'] in ["Changing Lanes", "Lane / Road Departure"]:
        return 2
    elif row['SV Pre-Crash Movement'] in ["Making Right Turn", "Making Left Turn"]:
        return 3
    elif row['SV Pre-Crash Movement'] in ["Backing", "Other, see Narrative", "Entering Traffic", "Parked", "Merging", "Unknown", "Parking Maneuver"]:
        return 4
    else:
        return 99

# Apply the function to create a new variable
df['SV_Pre_Crash_M'] = df.apply(new_var, axis=1)

# Replace blanks with 99
df['SV_Pre_Crash_M'].replace(" ", 99, inplace=True)


In [25]:
def new_var(row):
    if row['Roadway Type'] == "Highway / Freeway":
        return 1
    elif row['Roadway Type'] == "Intersection":
        return 2
    elif row['Roadway Type'] == "Street":
        return 3
    elif row['Roadway Type'] == "Parking Lot":
        return 4
    elif row['Roadway Type'] == "Unknown" or row['Roadway Type'] == "Traffic Circle":
        return 5
    else:
        return 0
    
df['Crash_Location'] = df.apply(new_var, axis=1)
df['Crash_Location'].replace(" ", 99, inplace=True)


In [26]:
# Create a new column for Driver / Operator Type
df['Driver Type'] = df['Driver / Operator Type']

# Replace values in new column based on conditions
df['Driver Type'] = df['Driver Type'].replace(['None', 'In-Vehicle (Commercial / Test)', 'Consumer', 'Other, see Narrative', 'Unknown'], 0)
df['Driver Type'] = df['Driver Type'].replace('Remote (Commercial / Test)', 1)
df['Driver Type'] = df['Driver Type'].replace('In-Vehicle and Remote (Commercial / Test)', 2)

# Replace blanks with 99
df['Driver Type'] = df['Driver Type'].replace('', 99)


In [27]:
# Create a new column for the crash with type
df['Crash With Type'] = 0

# Assign values based on 'Crash With' column
df.loc[df['Crash With'].isin(['Passenger Car', 'SUV']), 'Crash With Type'] = 1
df.loc[df['Crash With'].isin(['Non-Motorist: Cyclist', 'Motorcycle', 'Non-Motorist: Pedestrian', 'Non-Motorist: Other']), 'Crash With Type'] = 2
df.loc[df['Crash With'] == 'Pickup Truck', 'Crash With Type'] = 3
df.loc[df['Crash With'] == 'Heavy Truck', 'Crash With Type'] = 4
df.loc[df['Crash With'].isin(['Other Fixed Object', 'Van', 'Other, see Narrative', 'First Responder Vehicle', 'Unknown', 'Animal', 'Bus', 'Pole / Tree']), 'Crash With Type'] = 5

# Replace blank values with 99
df['Crash With Type'].replace('', 99, inplace=True)


In [28]:
import numpy as np

# Replace blanks with 99
df['Narrative'].replace(" ", "99", inplace=True)

# Create new variable 'Disengaged' based on presence of keywords
df['Disengaged'] = np.where(df['Narrative'].str.contains('disengage|disengaged', case=False, na=False), 1, 0)


In [34]:
# Create a new variable for Roadway Surface
df['Roadway Surface Value'] = df['Roadway Surface'].map({'Dry': 0, 'Wet': 1, 'Snow / Slush / Ice': 2, 'Unknown': 2})

# Replace blank values with 99
df['Roadway Surface Value'] = df['Roadway Surface Value'].replace('', 99)

In [37]:
#Injury vs Non-injury Var
df['injuryVSnoninjury'] = df['Highest Injury Severity'].replace({'No Injuries Reported': 0,
                                                        'Minor': 1,
                                                        'Moderate': 1,
                                                        'Serious': 1,
                                                        'Unknown': 98})

In [38]:
# Save the cleaned DataFrame to a new CSV file
df.to_csv('df_ADS_pyClean1.csv', index=False)

In [44]:
import geopy
from geopy.geocoders import Nominatim

# Initialize Nominatim API geolocator
geolocator = Nominatim(user_agent="my-app")

# Define function to get latitude and longitude for a given city
def get_lat_lon(city):
    location = geolocator.geocode(city)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Apply the function to the "City" column and create new columns for latitude and longitude
df[['Latitude1', 'Longitude1']] = df['City'].apply(lambda x: pd.Series(get_lat_lon(x)))


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=San+Francisco&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x000001DB5D3AEC70>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))

In [42]:
df.head()

,Report ID,Report Version,Reporting Entity,Report Type,Report Month,Report Year,Report Submission Date,VIN,VIN - Unknown,Serial Number,...,SVc_Sides,Weather,Lighting_recode,SV_Pre_Crash_M,Crash_Location,Driver Type,Crash With Type,Disengaged,Roadway Surface Value,injuryVSnoninjury
0,30270-5085,2,Waymo LLC,Other Update,NaN,NaN,MAR-2023,SADHW2S11M1,,NaN,...,0,0,0,1,2,0,4,0,0,0
1,30270-5081,1,Waymo LLC,Monthly,2.0,2023.0,MAR-2023,SADHW2S15M1,,NaN,...,0,0,1,0,2,0,5,0,0,0
2,30270-5085,1,Waymo LLC,Monthly,2.0,2023.0,MAR-2023,SADHW2S11M1,,NaN,...,0,0,0,1,2,0,4,0,0,0
3,30270-5082,1,Waymo LLC,Monthly,2.0,2023.0,MAR-2023,SADHW2S15M1,,NaN,...,0,0,0,0,3,0,1,0,0,0
4,30270-5083,1,Waymo LLC,Monthly,2.0,2023.0,MAR-2023,SADHW2S17M1,,NaN,...,1,2,1,3,2,0,1,0,1,0
